## Master in Business Analytics &amp; Big Data - Master's thesis
### "Sales prediction and anomaly detection in times of COVID-19 using data from the stock market and the international press”

Full PDF can be read at this link:
<br>
https://www.linkedin.com/posts/sergio-carrasco-sanchez_final-masters-project-activity-6746471092820475904-YLCE

## Data loading and cleaning
### Sales data load

In [64]:
!pip install --upgrade pip

In [65]:
!pip install pandas

import pandas as pd

We proceed to load sales data from CSV file.

In [66]:
sales_data_df = pd.read_csv('./datasets/orders.csv')

In [67]:
sales_data_df

,id,date,user,payment_method,total,subtotal,discount,shipping_costs
0,1,2014-05-02 18:32:01,10614,bank_transfer,7.85,2.95,0.0,4.9
1,2,2014-05-02 19:01:55,10614,bank_transfer,8.19,3.29,0.0,4.9
2,3,2014-05-05 15:39:38,10615,cash_on_delivery,10.65,5.75,0.0,4.9
3,7,2014-05-12 14:48:47,10619,bank_transfer,7.39,2.49,0.0,4.9
4,8,2014-05-12 14:52:46,10619,cash_on_delivery,8.00,3.10,0.0,4.9
...,...,...,...,...,...,...,...,...
119375,128481,2020-10-31 21:16:04,68077,cash_on_delivery,13.25,8.35,0.0,4.9
119376,128482,2020-10-31 21:30:12,68078,cash_on_delivery,16.80,11.90,0.0,4.9
119377,128483,2020-10-31 22:00:58,41554,credit_card,52.86,47.96,0.0,4.9
119378,128484,2020-10-31 22:13:14,64540,paypal,23.25,18.35,0.0,4.9


We remove the least interesting columns.

In [68]:
sales_data_df = sales_data_df.drop(['id', 'user', 'payment_method', 'subtotal', 'discount', 'shipping_costs'], axis=1)

In [69]:
sales_data_df = sales_data_df.rename({'total': 'sales'}, axis=1)

In [70]:
sales_data_df

,date,sales
0,2014-05-02 18:32:01,7.85
1,2014-05-02 19:01:55,8.19
2,2014-05-05 15:39:38,10.65
3,2014-05-12 14:48:47,7.39
4,2014-05-12 14:52:46,8.00
...,...,...
119375,2020-10-31 21:16:04,13.25
119376,2020-10-31 21:30:12,16.80
119377,2020-10-31 22:00:58,52.86
119378,2020-10-31 22:13:14,23.25


We group the sales by day.

In [71]:
def daily_sales(data):
    data = data.copy()
    data.date = data.date.apply(lambda x: str(x)[:-9])
    data = data.groupby('date')['sales'].sum().reset_index()
    data.date = pd.to_datetime(data.date)
    return data

In [72]:
daily_sales_df = daily_sales(sales_data_df)

In [73]:
daily_sales_df

,date,sales
0,2014-05-02,16.04
1,2014-05-05,10.65
2,2014-05-12,53.54
3,2014-05-13,135.45
4,2014-05-15,144.89
...,...,...
2337,2020-10-27,4016.97
2338,2020-10-28,4641.49
2339,2020-10-29,3006.00
2340,2020-10-30,2181.25


In [74]:
daily_sales_df = daily_sales_df.set_index(daily_sales_df.date)

In [75]:
daily_sales_df = daily_sales_df.drop('date', axis=1)

In [76]:
daily_sales_df

,sales
date,
2014-05-02,16.04
2014-05-05,10.65
2014-05-12,53.54
2014-05-13,135.45
2014-05-15,144.89
...,...
2020-10-27,4016.97
2020-10-28,4641.49
2020-10-29,3006.00


### Nasdaq Composite Index Data Load

We import the  Composite index data from the same time period as the sales data.

In [77]:
!pip install yfinance

import yfinance as yf

In [78]:
min_date = min(daily_sales_df.index)
max_date = max(daily_sales_df.index)

In [79]:
min_date

Timestamp('2014-05-02 00:00:00')

In [80]:
max_date

Timestamp('2020-10-31 00:00:00')

In [81]:
ticker = '^IXIC'
ticker_name = 'NASDAQ Composite'

In [82]:
stock_data_df = yf.download(ticker, start=min_date, end=max_date)

[*********************100%***********************]  1 of 1 completed


In [83]:
stock_data_df.reset_index(inplace=True)

In [84]:
stock_data_df['Date'] = stock_data_df['Date'].dt.date

In [85]:
stock_data_df = stock_data_df.set_index(stock_data_df['Date'])

In [86]:
stock_data_df = stock_data_df.drop(['Date'], axis=1)

In [87]:
stock_data_df.index.name = 'date'

In [88]:
stock_data_df

,Open,High,Low,Close,Adj Close,Volume
date,,,,,,
2014-05-02,4138.629883,4145.060059,4115.890137,4123.899902,4123.899902,1844790000
2014-05-05,4099.250000,4138.339844,4086.350098,4138.060059,4138.060059,1561170000
2014-05-06,4128.220215,4132.459961,4080.760010,4080.760010,4080.760010,1850610000
2014-05-07,4085.489990,4091.030029,4021.050049,4067.669922,4067.669922,2486030000
2014-05-08,4053.280029,4109.200195,4039.909912,4051.500000,4051.500000,2411940000
...,...,...,...,...,...,...
2020-10-26,11440.639648,11545.629883,11221.059570,11358.940430,11358.940430,3186950000
2020-10-27,11409.339844,11465.059570,11361.860352,11431.349609,11431.349609,3079530000
2020-10-28,11230.900391,11249.950195,10999.070312,11004.870117,11004.870117,3912580000


We combine the sales data and the index data in the same dataframe.

In [89]:
daily_sales_df = pd.merge(left=daily_sales_df,
                          right=stock_data_df,
                          left_index=True,
                          right_index=True,
                          how='inner')

In [90]:
daily_sales_df = daily_sales_df.rename({'Open':'stock_open',
                                        'High':'stock_high',
                                        'Low':'stock_low',
                                        'Close':'stock_close',
                                        'Adj Close':'stock_adjclose',
                                        'Volume':'stock_volume'},
                                       axis=1)

In [91]:
daily_sales_df = daily_sales_df[['sales',
                                 'stock_open',
                                 'stock_high',
                                 'stock_low',
                                 'stock_close',
                                 'stock_adjclose',
                                 'stock_volume']]

In [92]:
daily_sales_df

,sales,stock_open,stock_high,stock_low,stock_close,stock_adjclose,stock_volume
date,,,,,,,
2014-05-02,16.04,4138.629883,4145.060059,4115.890137,4123.899902,4123.899902,1844790000
2014-05-05,10.65,4099.250000,4138.339844,4086.350098,4138.060059,4138.060059,1561170000
2014-05-12,53.54,4092.840088,4146.540039,4092.090088,4143.859863,4143.859863,1880020000
2014-05-13,135.45,4144.899902,4155.129883,4128.009766,4130.169922,4130.169922,1923480000
2014-05-15,144.89,4096.529785,4098.250000,4035.959961,4069.290039,4069.290039,2083030000
...,...,...,...,...,...,...,...
2020-10-26,4176.29,11440.639648,11545.629883,11221.059570,11358.940430,11358.940430,3186950000
2020-10-27,4016.97,11409.339844,11465.059570,11361.860352,11431.349609,11431.349609,3079530000
2020-10-28,4641.49,11230.900391,11249.950195,10999.070312,11004.870117,11004.870117,3912580000
